Note: this notebook requires a `.coveragerc` file next to it with the following content:

```ini
[run]
plugins =
    ipytest.cov
```

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
import json
import linecache

from pathlib import Path


def get_executed_lines(data, filename):
    return [
        linecache.getline(filename, line).strip()
        for line in sorted(data["files"][filename]["executed_lines"])
    ]


def func(x):
    if x % 2 == 0:
        return "even"

    else:
        return "odd"


# delete generated files
Path(".coverage").unlink(missing_ok=True)
Path("coverage.json").unlink(missing_ok=True)

In [ ]:
%%ipytest --cov --cov-report=json

def test():
    assert func(0) == "even"
    assert func(1) == "odd"
    assert func(2) == "even"
    assert func(3) == "odd"

In [ ]:
assert Path(".coverage").exists()
assert Path("coverage.json").exists()

with open("coverage.json", "rt") as fobj:
    data = json.load(fobj)

assert get_executed_lines(data, func.__code__.co_filename) == [
    "if x % 2 == 0:", 
    "return \"even\"", 
    "return \"odd\"",
]

assert get_executed_lines(data, test.__code__.co_filename) == [
    "assert func(0) == \"even\"",
    "assert func(1) == \"odd\"",
    "assert func(2) == \"even\"",
    "assert func(3) == \"odd\"",
]